In [1]:
import math
import random
from collections import defaultdict
import time
from itertools import permutations

class M3System:
    def __init__(self, A, B, C, D, E, modulus):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E
        self.modulus = modulus
        self.identity_element = M3Element(0, 0, 0, self) 

    def __str__(self):
        return f"M3System(A={self.A}, B={self.B}, C={self.C}, D={self.D}, E={self.E}, modulus={self.modulus})"

    def get_identity(self):
        return self.identity_element

class M3Element:
    def __init__(self, x0, x1, x2, system):
        self.x0 = x0 % system.modulus
        self.x1 = x1 % system.modulus
        self.x2 = x2 % system.modulus
        self.system = system

    def __mul__(self, other):
        if not isinstance(other, M3Element) or self.system != other.system:
            raise ValueError("Can only multiply M3Element with another M3Element from the same system.")

        s = self.system
        new_x0 = (self.x0 + other.x0 + self.x0 * other.x0 +
                  s.A * self.x1 * other.x1 +
                  s.C * self.x2 * other.x1 +
                  s.B * self.x2 * other.x2) % s.modulus

        new_x1 = (self.x1 + other.x1 + self.x1 * other.x0 +
                  self.x0 * other.x1 +
                  s.D * self.x1 * other.x1 +
                  s.E * self.x1 * other.x2) % s.modulus

        new_x2 = (self.x2 + other.x2 + self.x2 * other.x0 +
                  self.x0 * other.x2 +
                  s.D * self.x2 * other.x1 +
                  s.E * self.x2 * other.x2) % s.modulus

        return M3Element(new_x0, new_x1, new_x2, s)

    def __pow__(self, exponent):
        if not isinstance(exponent, int) or exponent < 0:
            raise ValueError("Exponent must be a non-negative integer.")
        if exponent == 0:
            return self.system.get_identity()
        
        result = self.system.get_identity() 
        base = self

        while exponent > 0:
            if exponent % 2 == 1:
                result = result * base
            base = base * base
            exponent //= 2
        return result

    def __eq__(self, other):
        if not isinstance(other, M3Element):
            return NotImplemented
        return (self.x0 == other.x0 and
                self.x1 == other.x1 and
                self.x2 == other.x2 and
                self.system == other.system)

    def __hash__(self):
        return hash((self.x0, self.x1, self.x2, hash(self.system)))

    def __str__(self):
        return f"[{self.x0}, {self.x1}, {self.x2}]"

    def __repr__(self):
        return self.__str__()

    def to_tuple(self):
        return (self.x0, self.x1, self.x2)

def is_prime(n):
    """Checks if a number is prime."""
    if n < 2: return False
    if n == 2 or n == 3: return True
    if n % 2 == 0 or n % 3 == 0: return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def get_primes_in_range(start, end):
    """Returns a list of prime numbers in a given range."""
    return [n for n in range(start, end + 1) if is_prime(n)]

def analyze_all_orbits(system):
    """
    Analyzes all possible orbits within the given M3System.
    It iterates through all elements in the system's space,
    finds their orbit lengths, and categorizes them.
    """
    modulus = system.modulus
    total_space_elements = modulus**3
    
    generated_cycles_lengths = []
    global_elements_in_orbits = set()
    
    # New list to store the starting elements of N^2-1 length cycles
    n_sq_minus_1_cycle_starts = [] 

    identity_element = system.get_identity()
    identity_tuple = identity_element.to_tuple()
    
    if identity_tuple not in global_elements_in_orbits:
        global_elements_in_orbits.add(identity_tuple)
        generated_cycles_lengths.append(1)

    for x0_val in range(modulus):
        for x1_val in range(modulus):
            for x2_val in range(modulus):
                current_seed = M3Element(x0_val, x1_val, x2_val, system)
                current_seed_tuple = current_seed.to_tuple()

                if current_seed_tuple in global_elements_in_orbits:
                    continue
                
                path = []
                path_dict = {}
                current_in_orbit = current_seed
                
                cycle_length = 0
                
                while True:
                    current_tuple = current_in_orbit.to_tuple()
                    
                    if current_tuple in path_dict:
                        cycle_start_index = path_dict[current_tuple]
                        cycle_length = len(path) - cycle_start_index
                        break
                    
                    if current_in_orbit == identity_element:
                        path.append(current_tuple)
                        cycle_length = len(path)
                        global_elements_in_orbits.add(identity_tuple)
                        break
                    
                    path_dict[current_tuple] = len(path)
                    path.append(current_tuple)
                    
                    current_in_orbit = current_in_orbit * current_seed

                generated_cycles_lengths.append(cycle_length)
                
                # Check if this cycle is of length N^2-1 and store its starting element
                if cycle_length == modulus * modulus - 1:
                    n_sq_minus_1_cycle_starts.append(current_seed_tuple)

                for p_elem_tuple in path:
                    global_elements_in_orbits.add(p_elem_tuple)
    
    total_found_orbits = len(generated_cycles_lengths)
    
    N_minus_1_count = 0
    N_sq_minus_1_count = 0
    N_minus_1_half_count = 0
    N_sq_minus_1_half_count = 0

    target_N_minus_1 = modulus - 1
    target_N_sq_minus_1 = modulus * modulus - 1
    target_N_minus_1_half = (modulus - 1) // 2 if (modulus - 1) % 2 == 0 else None
    target_N_sq_minus_1_half = (modulus * modulus - 1) // 2 if (modulus * modulus - 1) % 2 == 0 else None

    for length in generated_cycles_lengths:
        if length == target_N_minus_1:
            N_minus_1_count += 1
        elif length == target_N_sq_minus_1:
            N_sq_minus_1_count += 1
        elif target_N_minus_1_half is not None and length == target_N_minus_1_half:
            N_minus_1_half_count += 1
        elif target_N_sq_minus_1_half is not None and length == target_N_sq_minus_1_half:
            N_sq_minus_1_half_count += 1
    
    if total_found_orbits > 0:
        share_N_minus_1 = N_minus_1_count / total_found_orbits
        share_N_sq_minus_1 = N_sq_minus_1_count / total_found_orbits
        share_N_minus_1_half = N_minus_1_half_count / total_found_orbits
        share_N_sq_minus_1_half = N_sq_minus_1_half_count / total_found_orbits
    else:
        share_N_minus_1 = share_N_sq_minus_1 = share_N_minus_1_half = share_N_sq_minus_1_half = 0

    return {
        "Modulus": modulus,
        "A": system.A, "B": system.B, "C": system.C, "D": system.D, "E": system.E,
        "Total_Space_Elements": total_space_elements,
        "Total_Unique_Elements_Visited": len(global_elements_in_orbits),
        "Total_Orbits_Found": total_found_orbits,
        f"Orbits_Length_N-1 ({target_N_minus_1})_Count": N_minus_1_count,
        f"Share_N-1": share_N_minus_1,
        f"Orbits_Length_N^2-1 ({target_N_sq_minus_1})_Count": N_sq_minus_1_count,
        f"Share_N^2-1": share_N_sq_minus_1,
        f"Orbits_Length_N-1/2 ({target_N_minus_1_half})_Count": N_minus_1_half_count if target_N_minus_1_half is not None else 0,
        f"Share_N-1/2": share_N_minus_1_half,
        f"Orbits_Length_N^2-1/2 ({target_N_sq_minus_1_half})_Count": N_sq_minus_1_half_count if target_N_sq_minus_1_half is not None else 0,
        f"Share_N^2-1/2": share_N_sq_minus_1_half,
        "N_sq_minus_1_Cycle_Starts": n_sq_minus_1_cycle_starts # Add the list of starting elements
    }


# --- Main Test Script for fixed A, B, C, D, E ---
if __name__ == "__main__":
    test_modulus = 61 # Fixed modulus for this specific test

    # Fixed values for all parameters
    FIXED_A = test_modulus // 2 + 1
    FIXED_B = test_modulus // 2
    FIXED_C = 1
    FIXED_D = 1
    FIXED_E = 2

    # Ensure fixed values are valid (non-negative and less than modulus)
    # A, B, C must be non-zero and less than modulus. D, E can be 0.
    if not (0 <= FIXED_A < test_modulus and
            0 <= FIXED_B < test_modulus and
            0 <= FIXED_C < test_modulus and
            0 <= FIXED_D < test_modulus and
            0 <= FIXED_E < test_modulus):
        raise ValueError(
            f"Fixed parameter values are not valid for modulus {test_modulus}."
        )
            
    print(f"Running a single test for modulus N = {test_modulus}")
    print(f"Fixed parameters: A={FIXED_A}, B={FIXED_B}, C={FIXED_C}, D={FIXED_D}, E={FIXED_E}")
    print(f"----------------------------------------------------------------------------------------------------")

    start_total_time = time.time()
    
    system = M3System(FIXED_A, FIXED_B, FIXED_C, FIXED_D, FIXED_E, test_modulus)
    result = analyze_all_orbits(system)
    
    end_total_time = time.time()
    total_execution_time = end_total_time - start_total_time

    # Print the results directly for this single test
    print(f"\n====================================================================================================")
    print(f"=== ANALYSIS FOR N = {test_modulus} with A={FIXED_A}, B={FIXED_B}, C={FIXED_C}, D={FIXED_D}, E={FIXED_E} ===")
    print(f"Total execution time: {total_execution_time:.2f} seconds.")
    print(f"====================================================================================================")

    print(f"Modulus N: {result['Modulus']}")
    print(f"Parameters (A,B,C,D,E): ({result['A']}, {result['B']}, {result['C']}, {result['D']}, {result['E']})")
    print(f"Total space elements (N^3): {result['Total_Space_Elements']}")
    print(f"Total unique elements visited: {result['Total_Unique_Elements_Visited']}")
    print(f"Total orbits found: {result['Total_Orbits_Found']}")
    print(f"----------------------------------------------------------------------------------------------------")
    
    # Prepare the dictionary keys as separate variables for cleaner access
    N_minus_1_key = f'Orbits_Length_N-1 ({result["Modulus"]-1})_Count'
    N_sq_minus_1_key = f'Orbits_Length_N^2-1 ({result["Modulus"]**2-1})_Count'
    N_minus_1_half_key = f'Orbits_Length_N-1/2 ({(result["Modulus"]-1)//2})_Count'
    N_sq_minus_1_half_key = f'Orbits_Length_N^2-1/2 ({(result["Modulus"]**2-1)//2})_Count'

    print(f"Number of orbits with length N-1 ({result['Modulus']-1}): {result.get(N_minus_1_key)}")
    print(f"Share of orbits with length N-1: {result['Share_N-1']:.4f}")
    print(f"Number of orbits with length N^2-1 ({result['Modulus']**2-1}): {result.get(N_sq_minus_1_key)}")
    print(f"Share of orbits with length N^2-1: {result['Share_N^2-1']:.4f}")

    if result.get(N_minus_1_half_key) is not None:
        print(f"Number of orbits with length (N-1)/2 ({(result['Modulus']-1)//2}): {result.get(N_minus_1_half_key)}")
        print(f"Share of orbits with length (N-1)/2: {result['Share_N-1/2']:.4f}")
    if result.get(N_sq_minus_1_half_key) is not None:
        print(f"Number of orbits with length (N^2-1)/2 ({(result['Modulus']**2-1)//2}): {result.get(N_sq_minus_1_half_key)}")
        print(f"Share of orbits with length (N^2-1)/2: {result['Share_N^2-1/2']:.4f}")

    # --- New output for N^2-1 cycle starting elements ---
    print("\n--- Starting elements of N^2-1 length cycles ---")
    if result["N_sq_minus_1_Cycle_Starts"]:
        for i, start_element in enumerate(result["N_sq_minus_1_Cycle_Starts"]):
            print(f"  {start_element} - {i+1}th cycle")
    else:
        print("  No N^2-1 length cycles found.")

Running a single test for modulus N = 61
Fixed parameters: A=31, B=30, C=1, D=1, E=2
----------------------------------------------------------------------------------------------------

=== ANALYSIS FOR N = 61 with A=31, B=30, C=1, D=1, E=2 ===
Total execution time: 0.55 seconds.
Modulus N: 61
Parameters (A,B,C,D,E): (31, 30, 1, 1, 2)
Total space elements (N^3): 226981
Total unique elements visited: 226981
Total orbits found: 5302
----------------------------------------------------------------------------------------------------
Number of orbits with length N-1 (60): 4495
Share of orbits with length N-1: 0.8478
Number of orbits with length N^2-1 (3720): 31
Share of orbits with length N^2-1: 0.0058
Number of orbits with length (N-1)/2 (30): 459
Share of orbits with length (N-1)/2: 0.0866
Number of orbits with length (N^2-1)/2 (1860): 9
Share of orbits with length (N^2-1)/2: 0.0017

--- Starting elements of N^2-1 length cycles ---
  (0, 0, 2) - 1th cycle
  (0, 1, 4) - 2th cycle
  (0, 1